In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
%matplotlib inline

In [2]:
pre = pd.read_csv('pre_0901.csv')

pre_df = pd.DataFrame(pre)
pre_df

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking_status,cancer,date,observation_date
0,10015010,0,59,NaN,25.65,3.0,NaN,1,2010-03-12,2006-01-01
1,10015010,0,61,NaN,24.57,3.0,NaN,1,2010-03-12,2008-01-01
2,10015010,0,63,1.0,23.88,1.0,NaN,1,2010-03-12,2010-01-01
3,10015010,0,65,2.0,24.51,1.0,NaN,1,2010-03-12,2012-01-01
4,10051558,0,83,NaN,21.79,1.0,NaN,1,2010-08-05,2010-01-01
...,...,...,...,...,...,...,...,...,...,...
31320,94370705,1,61,NaN,17.40,1.0,1.0,1,2012-10-20,2011-01-01
31321,94370705,1,62,NaN,17.80,1.0,NaN,1,2012-10-20,2012-01-01
31322,94370705,1,63,1.0,18.83,1.0,1.0,1,2012-10-20,2013-01-01
31323,94406889,1,52,1.0,28.89,1.0,1.0,1,2011-02-28,2011-01-01


In [3]:
# 위암의 경우, 위암 최초 진단 직전 건강검진 기록만 남기기로 함. 
# 위암 진단 전의 건강 검진이 없는 경우, 추후 삭제

new_c = pd.DataFrame(columns=['person_id', 'gender', 'age', 'EGD', 'bmi', 'alcohol_habit','smoking_status', 'cancer', 'date', 'observation_date'])
persons=list(set(pre_df['person_id'].tolist()))

for p in persons:
    days=[]
    person_data = pre_df[pre_df['person_id']==p]
    
    for i in range(len(person_data)):
        cancer_date = pd.to_datetime(person_data.iloc[i,8]) # date
        check_date = pd.to_datetime(person_data.iloc[i,9]) #observation_date
        day = (cancer_date-check_date).days
        if day>0:
            days.append(99999)
        else:
            days.append(abs(day))
    min_date = days.index(min(days))
    #person_data.iloc[min_date,15]=cancer_date.year
    new_c = new_c.append(person_data.iloc[min_date,:])

In [4]:
print(len(persons))
print(len(new_c))

9081
9081


In [5]:
# 위암 이전 건강 검진 없는 경우, 삭제

new_c['date'] = pd.to_datetime(new_c['date'])
new_c['observation_date'] = pd.to_datetime(new_c['observation_date'])
new_c['diff'] = (new_c['date']-new_c['observation_date'])

In [6]:
new_c['diff'] = new_c['diff'].astype('str')

In [7]:
new_c

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking_status,cancer,date,observation_date,diff
814,15007747,1,53,NaN,17.26,1.0,1.0,1,2006-04-17,2013-01-01,-2451 days
2787,22741001,0,75,NaN,20.58,2.0,NaN,1,2012-08-18,2003-01-01,3517 days
30027,89653264,0,50,NaN,23.88,1.0,1.0,1,2007-02-01,2005-01-01,761 days
1090,16154646,0,51,1.0,27.97,1.0,1.0,1,2011-05-04,2011-01-01,123 days
15527,58294297,0,59,NaN,24.39,2.0,NaN,1,2011-01-27,2002-01-01,3313 days
...,...,...,...,...,...,...,...,...,...,...,...
14209,55640047,0,81,NaN,18.20,1.0,1.0,1,2009-12-07,2011-01-01,-390 days
2292,20774899,0,49,NaN,18.90,1.0,NaN,1,2010-05-28,2011-01-01,-218 days
26603,82345971,1,54,NaN,20.44,1.0,1.0,1,2004-04-01,2005-01-01,-275 days
29529,88539129,0,67,NaN,25.71,4.0,NaN,1,2013-06-03,2004-01-01,3441 days


In [8]:
index = []
for i in range(len(new_c)):
    if new_c.iloc[i,10][0]!='-':
        index.append(i)

In [10]:
new_c1 = new_c.iloc[index,:]

In [11]:
new_c1

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking_status,cancer,date,observation_date,diff
2787,22741001,0,75,NaN,20.58,2.0,NaN,1,2012-08-18,2003-01-01,3517 days
30027,89653264,0,50,NaN,23.88,1.0,1.0,1,2007-02-01,2005-01-01,761 days
1090,16154646,0,51,1.0,27.97,1.0,1.0,1,2011-05-04,2011-01-01,123 days
15527,58294297,0,59,NaN,24.39,2.0,NaN,1,2011-01-27,2002-01-01,3313 days
25823,80871454,0,38,NaN,28.31,1.0,NaN,1,2009-12-08,2004-01-01,2168 days
...,...,...,...,...,...,...,...,...,...,...,...
380,12746702,0,69,2.0,23.95,1.0,1.0,1,2002-03-28,2002-01-01,86 days
19389,65929166,0,50,NaN,22.91,2.0,NaN,1,2009-07-17,2009-01-01,197 days
28127,85131223,1,67,NaN,31.35,1.0,1.0,1,2012-03-04,2003-01-01,3350 days
16939,60915675,0,63,NaN,21.26,3.0,NaN,1,2008-06-10,2004-01-01,1622 days


In [12]:
#new_c1.to_csv('notunder_0901.csv',mode='w',index=False)